In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import multiprocessing
from torch import autograd
from fastai.conv_learner import *
from fastai.transforms import TfmType
from fasterai.transforms import *
from fasterai.images import *
from fasterai.dataset import *
from fasterai.visualize import *
from fasterai.callbacks import *
from fasterai.loss import *
from fasterai.modules import *
from fasterai.wgan import *
from fasterai.generators import *
from fastai.torch_imports import *
from pathlib import Path
from itertools import repeat
import tensorboardX
torch.cuda.set_device(2)
plt.style.use('dark_background')
torch.backends.cudnn.benchmark=True


In [3]:
DATA_PATH = Path('data/imagenet/ILSVRC/Data/CLS-LOC')
TRAIN_SOURCE_PATH = DATA_PATH/'train'
proj_id = 'bw2color'
TENSORBOARD_PATH = Path('data/tensorboard/Unet_Plus')

dpath = DATA_PATH/(proj_id + '_823_D.h5')
gpath = DATA_PATH/(proj_id + '_823_G.h5')
keep_pct=0.2
bs=4
sz=256
wd=1e-7
lr=3e-3
lrs = np.array([lr/100,lr/10,lr])
x_tfms = [BlackAndWhiteTransform()]
torch.backends.cudnn.benchmark=True

## Training

In [4]:
netG = Unet34(nf_factor=2, self_attention=True, sn=True, leakyReLu=False).cuda()

In [5]:
dataLoader = ImageGenDataLoader(sz=sz, bs=bs, path=TRAIN_SOURCE_PATH, keep_pct=keep_pct, x_tfms=x_tfms)

In [6]:
md = dataLoader.get_model_data()

In [7]:
#netGVis = ModelVisualizationHook(TENSORBOARD_PATH, netG, 'netG')
unet = LearnerGenModuleWrapper(netG, 'unet')
#load_model(netG, gpath)
learn = ConvLearner(md, unet)
imgGenVis = ImageGenVisualizationCallback(TENSORBOARD_PATH, netG, md, 'learner')
learn.metrics = []
learn.opt_fn=optim.Adam
#learn.crit = F.mse_loss #(turns sepia/blurry)
learn.crit = FeatureLoss()

In [8]:
learn.freeze_to(-1)

In [9]:
#learn.lr_find(1e-4, 1e1, wds=wd, linear=False)

In [10]:
#learn.sched.plot(n_skip=0, n_skip_end=0)

In [ ]:
learn.fit(lrs, 1, cycle_len=1, wds=wd, use_clr_beta=(1,10,0.9,0.9), callbacks=[imgGenVis])

/media/jason/Projects/Deep Learning/Deep Learning Projects/Jupyter Experiments/1_BlackWhiteToColor/fasterai/generators.py:160: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if h<target_h or w<target_w:


Failed to generate graph for model: %465 : Long() = onnx::Constant[value={2}]()
 has empty scope name. Note that there's an outstanding issue with scopes being addressed here:  https://github.com/pytorch/pytorch/pull/12400


/home/jason/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/onnx/symbolic.py:130: UserWarning: ONNX export failed on dim because ONNX and PyTorch use different strategies to split the input. not supported
  warnings.warn("ONNX export failed on " + op + " because " + msg + " not supported")


  5%|▌         | 3193/60892 [35:19<10:38:14,  1.51it/s, loss=0.0222]

In [ ]:
learn.freeze_to(0)

In [ ]:
learn.fit(lrs/10, 1, cycle_len=1, wds=wd, use_clr_beta=(10,10,0.95,0.85), callbacks=[imgGenVis])